# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
T. Kalda  ->  T. Kalda  |  ['T. Kalda']
G. Green  ->  G. Green  |  ['G. Green']
S. Kraus  ->  S. Kraus  |  ['S. Kraus']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']


S. Bosman  ->  S. Bosman  |  ['S. Bosman']
Arxiv has 69 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2512.02095


extracting tarball to tmp_2512.02095... done.
Retrieving document from  https://arxiv.org/e-print/2512.02115


extracting tarball to tmp_2512.02115... done.


T. Kalda  ->  T. Kalda  |  ['T. Kalda']


...

/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2512.02116/kappavel.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'ANC/discussion_tab' from 'tmp_2512.02116/ANC/discussion_tab.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'ANC/orbit_fit_table' from 'tmp_2512.02116/ANC/orbit_fit_table.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.25/x6

extracting tarball to tmp_2512.02802...

 done.
Retrieving document from  https://arxiv.org/e-print/2512.02997
extracting tarball to tmp_2512.02997...

 done.


Issues with the citations
repeated bibliography entry: Gracia-Carpio2011


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.02115-b31b1b.svg)](https://arxiv.org/abs/2512.02115) | **Determining the Milky Way gravitational potential without selection functions**  |
|| <mark>T. Kalda</mark>, <mark>G. Green</mark> |
|*Appeared on*| *2025-12-03*|
|*Comments*| *13 pages, 5 figures*|
|**Abstract**|            Selection effects, such as interstellar extinction and varying survey depth, complicate efforts to determine the gravitational potential - and thus the distribution of baryonic and dark matter - throughout the Milky Way galaxy using stellar kinematics. We present a new variant of the "Deep Potential" method of determining the gravitational potential from a snapshot of stellar positions and velocities that does not require any modeling of spatial selection functions. Instead of modeling the full six-dimensional phase-space distribution function $f\left(\vec{x},\vec{v}\right)$ of observed kinematic tracers, we model the conditional velocity distribution $p\left(\vec{v}\mid\vec{x}\right)$, which is unaffected by a purely spatial selection function. We simultaneously learn the gravitational potential $\Phi\left(\vec{x}\right)$ and the underlying spatial density of the entire tracer population $n\left(\vec{x}\right)$ - including unobserved stars - using the collisionless Boltzmann equation under the stationarity assumption. The advantage of this method is that unlike the spatial selection function, all of the quantities we model, $p\left(\vec{v}\mid\vec{x}\right)$, $\Phi\left(\vec{x}\right)$, and $n\left(\vec{x}\right)$, typically vary smoothly in both position and velocity. We demonstrate that this "conditional" Deep Potential method is able to accurately recover the gravitational potential in a mock dataset with a complex three-dimensional dust distribution that imprints fine angular structure on the selection function. Because we do not need to model the spatial selection function, our new method can effectively scale to large, complex datasets while using relatively few parameters, and is thus well suited to Gaia data.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.02997-b31b1b.svg)](https://arxiv.org/abs/2512.02997) | **MIRI spectrophotometry of GN-z11: Detection and nature of an optical red continuum component**  |
|| A. C. Gómez, et al. -- incl., <mark>S. Bosman</mark> |
|*Appeared on*| *2025-12-03*|
|*Comments*| *14 pages, 5 figures. Accepted in A&A*|
|**Abstract**|            We present new MIRI F560W, F770W and F1000W imaging of GN-z11, extending the previous rest-frame coverage from 0.38 to 0.86$\mu$m. We report significant detections (14$\sigma$) in the F560W and F770W images, and a marginal detection (3.2$\sigma$) in F1000W. Here, we analyse its SED combining new MIRI imaging data with archival NIRSpec/Prism and MRS spectroscopy, and NIRCam imaging. The continuum emission shows a flat energy distribution, in f$_\nu$, up to 0.5$\mu$m, compatible with the presence of a mixed stellar population of young (4$\pm$1 Myr) and mature (63$\pm$23 Myr) stars that also account for the [O III], H$\beta$ and H$\alpha$ emission lines. The continuum at rest-frame 0.66$\mu$m shows a 36$\pm$3% flux excess above the predicted flux for a mixed stellar population, pointing to the presence of an additional source contributing at these wavelengths. This excess increases to 91$\pm$28% at rest-frame 0.86$\mu$m, although with a large uncertainty due to the marginal detection in the F1000W filter. We consider that hot dust emission in the dusty torus around a type 2 AGN could be responsible for the observed excess. Alternatively, this excess could be due to hot dust emission or to a photoluminiscence dust process (Extended Red Emission, ERE) under the extreme UV radiation field, as observed in some local metal-poor galaxies and in young compact starbursts. The presence of a type 1 AGN is not supported by the observed SED since high-z QSOs contribute at wavelengths above rest-frame 1$\mu$m, and an additional ad-hoc red source would be required to explain the observed flux excess at 0.66 and 0.86$\mu$m. Additional deep MIRI imaging covering the rest-frame near-IR are needed to confirm the flux detection at 10$\mu$m with higher significance, and to discriminate between the different hot dust emission in the extreme starburst and AGN scenarios with MIRI imaging at longer wavelengths.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.02802-b31b1b.svg)](https://arxiv.org/abs/2512.02802) | **SDSS-V LVM: Resolving Physical Conditions in the Trifid Nebula**  |
|| N. Sattler, et al. -- incl., <mark>K. Kreckel</mark> |
|*Appeared on*| *2025-12-03*|
|*Comments*| *accepted for publication in A&A*|
|**Abstract**|            The chemical abundance of the interstellar medium sets the initial conditions for star formation and provides a probe of chemical galaxy evolution models. However, unresolved inhomogeneities in the electron temperature can lead to a systematic underestimation of the abundances. We aim to directly test this effect. We use the SDSS-V Local Volume Mapper to spatially map the physical conditions of the Trifid Nebula (M 20), a Galactic H II region ionized by a single mid-type O star, at 0.24 pc resolution. We exploit various emission lines (e.g., Hydrogen recombination lines and collisionally excited lines, including also faint auroral lines) and compute spatially resolved maps of [O II] and [S II] electron densities; [N II], [O II], [S II], [S III] electron temperatures; and the ionic oxygen abundances. We find internal variations of electron density that result from the ionization front, along with a negative radial gradient. However, we do not find strong gradients or structures in the electron temperature and the total oxygen abundance, making the Trifid Nebula a relatively homogeneous H II region at the observed spatial scale. We compare these spatially resolved properties with equivalent integrated measurements of the Trifid Nebula and find no significant variations between integrated and spatially resolved conditions. This isolated H II region, ionized by a single O-star, represents a test case of an ideal Strömgren sphere. The physical conditions in the Trifid Nebula behave as expected, with no significant differences between integrated and resolved measurements.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.02095-b31b1b.svg)](https://arxiv.org/abs/2512.02095) | **The DREAMS Project: Disentangling the Impact of Halo-to-Halo Variance and Baryonic Feedback on Milky Way Satellite Galaxies**  |
|| J. C. Rose, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-12-03*|
|*Comments*| *16 pages, 5 figures, 2 tables, and 2 appendices with an additional 4 figures*|
|**Abstract**|            We analyze the properties of satellite galaxies around 1,024 Milky Way-mass hosts from the DREAMS Project, simulated within a $\Lambda$CDM cosmology. Utilizing the TNG galaxy-formation model, the DREAMS simulations incorporate both baryonic physics and cosmological uncertainties for a large sample of galaxies with diverse environments and formation histories. We investigate the relative impact of the physical uncertainty from the galaxy-formation model on predicted satellite properties using four metrics: the satellite stellar mass function, radial distribution, inner slope of dark matter density profile, and stellar half-light radius. We compare these predictions to observations from the SAGA Survey and the DREAMS N-body simulations and find that uncertainties from baryonic physics modeling are subdominant to the scatter arising from halo-to-halo variance. Where baryonic modeling does affect satellites, the supernova wind energy has the largest effect on the satellite properties that we investigate. Specifically, increased supernova wind energy suppresses the stellar mass of satellites and results in more extended stellar half-light radii. The adopted wind speed has only a minor impact, and other astrophysical and cosmological parameters show no measurable effect. Our findings highlight the robustness of satellite properties against uncertainties in baryonic physics modeling.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.02116-b31b1b.svg)](https://arxiv.org/abs/2512.02116) | **Hidden in Plain Sight II: Characterizing the luminous companion to Kappa Velorum with VLTI/GRAVITY**  |
|| D. M. Rowan, <mark>S. Kraus</mark>, T. A. Thompson |
|*Appeared on*| *2025-12-03*|
|*Comments*| *16 pages, 9 figures. Submitted to PASP*|
|**Abstract**|            Kappa Velorum (Markeb, HD 81188) is one of the brightest stars in the Southern sky and has long been known to be a single-lined spectroscopic binary. The binary mass function is large, $f(M)=1.15\ M_\odot$, suggesting that the bright (V=2.5) B2IV star may host a dark, compact object companion. We use VLTI GRAVITY observations to definitively test this possibility by directly resolving the binary. We detect a main sequence B star companion and rule out the compact object scenario. By combining the relative astrometric orbit and archival radial velocities, we report an updated precise characterization of the orbit (period $P=116.795\pm0.002$ d, eccentricity $e=0.1764\pm0.0004$, inclination $i=74.04\pm0.01^{\circ}$) and estimate the masses of the B stars. Using the original Hipparcos parallax measurement $\varpi = 6.05\pm0.48$ mas, we find $M_1 = 10^{+4}_{-2}\ M_\odot$ and $M_2 = 6.9\pm1.0\ M_\odot$. The uncertainties on the masses are primarily driven by the uncertain parallax, which we find is likely biased by the orbital motion. We use an archival UVES spectrum and MIST evolutionary tracks to refine our mass estimates. Finally, we discuss how interferometry and high-contrast imaging may be used to characterize other candidate star+compact object binaries, including those that will be discovered with Gaia DR4, as part of a larger effort to uncover the hidden population of black holes in the Milky Way.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.02167-b31b1b.svg)](https://arxiv.org/abs/2512.02167) | **Impact of Nuclear Reaction Rate Uncertainties on Type I X-ray Burst Nucleosynthesis: A Monte Carlo Study**  |
|| Q. Wang, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-12-03*|
|*Comments*| *23 pages, 13 figures, 2 tables*|
|**Abstract**|            To investigate the impact of nuclear reaction rate uncertainties on type I X-ray burst nucleosynthesis, comprehensive Monte Carlo simulations are performed with temperature-independent and -dependent variations in reaction rates using the REACLIB and STARLIB libraries, respectively. A total of 1,711 $(p, \gamma)$, $(p, \alpha)$, $(\alpha, p)$, and $(\alpha, \gamma)$ reactions are varied simultaneously, along with their inverse reactions, via detailed balance. For the first time, it is found that Monte Carlo sampling with larger perturbations to these reaction rates may lead to multi-peaked abundance distributions for some isotopes. These multi-peak structures arise not only from coupled reactions but also, in some cases, from single reactions. Our study also confirmed previously identified key reactions and provides more robust lists. These reactions deserve priority consideration in future study.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2512.02115/./mock_summary_xy_plane.png', 'tmp_2512.02115/./model_comparison_marginals.png', 'tmp_2512.02115/./selfn_mw_comparison.png']
copying  tmp_2512.02115/./mock_summary_xy_plane.png to _build/html/
copying  tmp_2512.02115/./model_comparison_marginals.png to _build/html/
copying  tmp_2512.02115/./selfn_mw_comparison.png to _build/html/
exported in  _build/html/2512.02115.md
    + _build/html/tmp_2512.02115/./mock_summary_xy_plane.png
    + _build/html/tmp_2512.02115/./model_comparison_marginals.png
    + _build/html/tmp_2512.02115/./selfn_mw_comparison.png
found figures ['tmp_2512.02997/./Color-color_test3.png', 'tmp_2512.02997/./SED_comp-templ_norm2.png', 'tmp_2512.02997/./SED_CIGALEvsSynth_fin_new.png']
copying  tmp_2512.02997/./Color-color_test3.png to _build/html/
copying  tmp_2512.02997/./SED_comp-templ_norm2.png to _build/html/
copying  tmp_2512.02997/./SED_CIGALEvsSynth_fin_new.png to _build/html/
exported in  _build/html/2512.02997.md
    + _build/html/

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\twocolswitch}[2]{#1}$
$\newcommand{\twocolswitch}[2]{#2}$</div>



<div id="title">

# Determining the Milky Way gravitational potential without selection functions

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.02115-b31b1b.svg)](https://arxiv.org/abs/2512.02115)<mark>Appeared on: 2025-12-03</mark> -  _13 pages, 5 figures_

</div>
<div id="authors">

<mark>T. Kalda</mark>, G. M. Green

</div>
<div id="abstract">

**Abstract:** Selection effects, such as interstellar extinction and varying survey depth, complicate efforts to determine the gravitational potential -- and thus the distribution of baryonic and dark matter -- throughout the Milky Way galaxy using stellar kinematics. We present a new variant of the "Deep Potential" method of determining the gravitational potential from a snapshot of stellar positions and velocities that does not require any modeling of spatial selection functions. Instead of modeling the full six-dimensional phase-space distribution function $f\left(\vec{x},\vec{v}\right)$ of observed kinematic tracers, we model the conditional velocity distribution $p\left(\vec{v}\mid\vec{x}\right)$ , which is unaffected by a purely spatial selection function. We simultaneously learn the gravitational potential $\Phi\left(\vec{x}\right)$ and the underlying spatial density of the entire tracer population $n\left(\vec{x}\right)$ -- including unobserved stars -- using the collisionless Boltzmann equation under the stationarity assumption. The advantage of this method is that unlike the spatial selection function, all of the quantities we model, $p\left(\vec{v}\mid\vec{x}\right)$ , $\Phi\left(\vec{x}\right)$ , and $n\left(\vec{x}\right)$ , typically vary smoothly in both position and velocity. We demonstrate that this "conditional" Deep Potential method is able to accurately recover the gravitational potential in a mock dataset with a complex three-dimensional dust distribution that imprints fine angular structure on the selection function. Because we do not need to model the spatial selection function, our new method can effectively scale to large, complex datasets while using relatively few parameters, and is thus well-suited to _Gaia_ data.

</div>

<div id="div_fig1">

<img src="tmp_2512.02115/./mock_summary_xy_plane.png" alt="Fig2" width="100%"/>

**Figure 2. -** Two-dimensional slices (the $z = 0$ plane) through the three-dimensional dust extinction density (left) and integrated extinction (as viewed from the origin; middle left), the resulting selection function (middle right), and the observed spatial distribution of stars (right). The observer is assumed to be at the origin. Dust extinction imprints fine angular structure on the selection function and the observed spatial distribution of stars, which can be difficult to capture with neural networks. (*fig:mock_data*)

</div>
<div id="div_fig2">

<img src="tmp_2512.02115/./model_comparison_marginals.png" alt="Fig3" width="100%"/>

**Figure 3. -** The gravitational densities ($\rho = \nabla^2 \Phi / 4 \pi G$) and their residuals (vs. the true densities), as recovered by the three variants of Deep Potential. **Top row:**$\ln \rho$ as a function of radius $r$, with the solid curves showing the median (in spherical shells) and the shaded envelopes enclosing the 16th to 84th percentiles. The dotted curves show the true density. **Second row:** Residuals $\Delta \ln \rho = \ln \rho_{\mathrm{model}} - \ln \rho_{\mathrm{true}}$ as a function of $r$. **Third row:** Residuals as a function of the true selection function $S\left(\vec{x}\right)$, illustrating the deterioration in the recovered density as the fraction of stars observed decreases. **Bottom row:** Residuals in the selection function $S\left(\vec{x}\right)$(as recovered by the "selection function" method) and true spatial density of tracers $n\left(\vec{x}\right)$(as recovered by the "conditional" method), each as a function of the true $S\left(\vec{x}\right)$. In the top three rows, the vertical axes are the same for the "selection function" and "conditional" methods. (*fig:model_marginals*)

</div>
<div id="div_fig3">

<img src="tmp_2512.02115/./selfn_mw_comparison.png" alt="Fig5" width="100%"/>

**Figure 5. -** Sky plots of the "selection function" method's prediction of the selection function $S(\vec x)$, the true selection function, and their fractional residuals on the surface of a sphere of radius \SI{4}{kpc}, centered on the mock observer. Areas with $S_\mathrm{true} < 0.01$ are masked. The differences can be attributed to shot noise in areas of low completeness, and to the difficulty of capturing fine angular structure with a neural network. Nevertheless, with a mock selection function with structure down to $\sim$0.25 kpc scales, our selection-function implementation performs well, albeit at the cost of large model sizes (here, $\sim$1.4 million parameters). (*fig:selfn_mw_comparison*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.02115"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\CII}[1]{[\ion{C}{II}] #1}$
$\newcommand{\OIII}[1]{[\ion{O}{III}] #1}$
$\newcommand{\SIII}[1]{[\ion{S}{III}] #1}$
$\newcommand{\Ha}{H\alpha\xspace}$
$\newcommand{\Hb}{H\beta\xspace}$
$\newcommand{\oddpm}[2]{\raisebox{0.5ex}{\tiny\substack{+#1 \ -#2}}}$
$\newcommand{\Mstar}{M_\mathrm{*}\xspace}$
$\newcommand{\Msyr}{ M_\odot yr^{-1}\xspace}$
$\newcommand{\arcs}{\arcsec\xspace}$
$\newcommand{\micron}{ \mum\xspace}$
$\newcommand{\Myr}{ Myr\xspace}$
$\newcommand{\Msun}{ M_\mathrm{\odot}\xspace}$
$\newcommand{\Lsun}{ L_\mathrm{\odot}\xspace}$
$\newcommand{\Zsun}{ Z_\mathrm{\odot}\xspace}$
$\newcommand{\Angs}{Å\xspace}$
$\newcommand{\ppccm}{cm^{-3}\xspace}$
$\newcommand{\kms}{ km s^{-1}\xspace}$
$\newcommand{\uJy}{ \muJy\xspace}$
$\newcommand{\mJy}{ mJy\xspace}$
$\newcommand{\nJy}{ nJy\xspace}$
$\newcommand{\ergs}{ erg s^{-1}\xspace}$
$\newcommand{\ergscm}{ erg s^{-1} cm^{-2}\xspace}$
$\newcommand{\ergscmAA}{ erg s^{-1} cm^{-2} Å^{-1}\xspace}$
$\newcommand{\EXP}[1]{ \times 10^{#1}}$
$\newcommand{\range}{ - }$
$\newcommand{\eg}{e.g., }$
$\newcommand{\ie}{i.e., }$
$\newcommand{\eqq}{ = }$
$\newcommand{\pmm}{ \pm }$
$\newcommand{\simi}{\sim }$
$\newcommand{\z}{z\xspace}$</div>



<div id="title">

# MIRI spectrophotometry of GN-z11: Detection and nature of an optical red continuum component

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.02997-b31b1b.svg)](https://arxiv.org/abs/2512.02997)<mark>Appeared on: 2025-12-03</mark> -  _14 pages, 5 figures. Accepted in A&A_

</div>
<div id="authors">

A. C. Gómez, et al. -- incl., <mark>S. Bosman</mark>

</div>
<div id="abstract">

**Abstract:** We present new MIRI F560W, F770W and F1000W imaging of the galaxy GN-z11 at a redshift of 10.603. We report a significant detection (14 $\sigma$ ) in the F560W and F770W images, and a marginal detection (3.2 $\sigma$ ) in the F1000W filter. The new MIRI observations cover the optical-red spectral range, and significantly extend previous NIRCam wavelength coverage from rest-frame 0.38 $\micron$ up to 0.86 $\micron$ . In this work, we analyse the spectral energy distribution (SED) combining this new MIRI imaging data with archival NIRSpec/Prism and MRS spectroscopy, and NIRCam imaging, i.e. covering the rest-frame 0.12 $\range$ 0.86 $\micron$ . New constraints such as the equivalent widths of the strong optical lines ( $\OIII{5008}$ , H $\beta$ and H $\alpha$ ) and the continuum emission at rest-frame 0.48 $\mu$ m, 0.66 $\mu$ m, and 0.86 $\mu$ m, free from emission line contributions, are presented. The continuum emission shows a flat energy distribution, in $f_\nu$ , up to 0.5 $\mu$ m, compatible with the presence of a mixed stellar population of young (4 $\pmm$ 1 Myr) and mature (63 $\pmm$ 23 Myr) stars that also account for the [ $\ion{O}{III}$ ] , $\Hb$ and $\Ha$ emission lines.The continuum at rest-frame 0.66 $\micron$ shows a 36 $\pmm$ 3 \% flux excess above the predicted flux for a mixed stellar population, pointing to the presence of an additional source contributing at these wavelengths. This excess increases to 91 $\pmm$ 28 \% at rest-frame 0.86 $\micron$ , although with a large uncertainty due to the marginal detection in the F1000W filter.We consider that hot dust emission in the dusty torus around a type 2 AGN could be responsible for the observed excess. Alternatively, this excess could be due to hot dust emission or to a photoluminiscence dust process (Extended Red Emission, ERE) under the extreme UV radiation field, as observed in some local metal-poor galaxies and in young compact starbursts. The presence of a type 1 AGN is not supported by the observed SED as the hot dust emission in luminous high- $\z$ QSOs contributes at wavelengths above rest-frame 1 $\micron$ , and an additional ad-hoc red source would be required to explain the observed flux excess at 0.66 and 0.86 $\micron$ . Additional deep MIRI imaging covering the rest-frame near-IR are needed to confirm the flux detection at 10 $\micron$ with higher significance, and to discriminate between the different hot dust emission in the extreme starburst and AGN scenarios with MIRI imaging at longer wavelengths.

</div>

<div id="div_fig1">

<img src="tmp_2512.02997/./Color-color_test3.png" alt="Fig2" width="100%"/>

**Figure 2. -** Continuum-only colour-colour diagrams. Black squares represent the GN-z11 colours based on the NIRCam and MIRI photometry after the subtraction of the emission lines contribution measured with NIRSpec and MRS ($\ie$ continuum-only colours) presented in Table \ref{tab:fluxes}. Continuum (dashed) lines display the continuum-only colours derived considering a stellar population colour-coded by their age assuming an instantaneous (continuum) SFH. These colours include the contribution from the stellar continuum calculated using the stellar models from BPASSv2.2 and its associated nebular continuum derived with Cloudy (see Sect. \ref{subsec:Cont-only}). Thin and thick lines represent the 0.05 and 0.2$\Zsun$ tracks, respectively. Orange, green and purple diamonds show the derived continuum-only colours from the composed $\z$$\simi$1 QSO (\citealt{Glikman+06,Selsing+16}, see Sect. \ref{subsec:SED_fitting}), the Seyfert 2 Mrk3 \citep{Spinelli+06} and a $\z$$\eqq$7.1 QSO \citep{Bosman2024,Bosman+25}. Red, blue and brown diamonds represent the values derived from a stacked spectra of $\simi$150 LRDs (\citealt{PerezGonzalez-LRD2024}, Pérez-González et al. in prep.), the COSMOS-Webb median-stacked LRD \citep{Akins+25} and the unique LRD known as 'The Cliff' \citep{deGraaff+25}, respectively. In the left and middle panels the brown diamond lie outside the colour range displayed. Black arrows in the lower right corners represent the vector magnitude for $A_\mathrm{V}$\eqq0.3 mag. (*fig:color*)

</div>
<div id="div_fig2">

<img src="tmp_2512.02997/./SED_comp-templ_norm2.png" alt="Fig3" width="100%"/>

**Figure 3. -** NIRCam and MIRI photometry compared with different red SED objects. Grey squares and black circles represent the NIRCam+MIRI observed and continuum-only (see Sect. \ref{subsec:Cont.-results}) fluxes, respectively. Coloured lines represent the SEDs for different AGNs types and high-$\z$ objects with optical-red colours presented in Sect. \ref{subsec:Cont.-results}. Red line represents the stacked spectra for $\simi$150 local and high-$\z$ LRDs (\citealt{PerezGonzalez-LRD2024}, Pérez-González et al. in prep.). Brown line displays the spectra from the exotic $\z$$\simi$3.5 LRD `The Cliff' while brown diamonds represent its re-normalised fluxes from \citet{deGraaff+25} at rest- 0.9 and 1.9$\micron$. Blue line shows the COSMOS-Webb median-stacked LRD SED model presented in \citet{Akins+25}. Orange and purple lines show the $\z$$\simi$1 QSO composite spectrum, created by combining the UV+optical and near-IR spectral templates from \citet{Selsing+16} and \citet{Glikman+06}, and the NIRSpec+MRS spectrum for the $\z$$\eqq$7.1 QSO J1120+0641 (\citealt{Bosman2024,Bosman+25}), respectively. The HST/STIS spectrum for the Sy2 Mrk3 presented by \citet{Spinelli+06} is displayed as a green line. All observed spectra have been redshifted to $\z$$\eqq$10.6 and normalised at 4800Å. (*fig:SED_templates*)

</div>
<div id="div_fig3">

<img src="tmp_2512.02997/./SED_CIGALEvsSynth_fin_new.png" alt="Fig5" width="100%"/>

**Figure 5. -** SED best-fit models considering the star formation and the different AGN scenarios. Top panel show the best-fit SEDs obtained with \texttt{CIGALE} assuming a type 1 and type 2 AGNs as blue and green lines, respectively. Red line displays the best-fit derived with \texttt{SYNTHESIZER-AGN} using a QSO template and an additional dusty torus. Black line and black circles represent the NIRSpec/Prism data presented in \citet{Bunker+23} and the NIRCam and MIRI photometry as derived in Sect. \ref{subsec:phot}, respectively. coloured squares show the modelled NIRCam and MIRI fluxes from the different best-fit models, while their residuals with the measured photometry are displayed below. Green, blue and red lines have been convolved to match the spectral resolution from NIRSpec/Prism ($\ie$ R=100). Middle panel shows the \texttt{CIGALE} best-fit models (in black) and the contributions of the nebular and stellar components (in blue) and the \citet{Fritz+06} torus emission (in brown). Dashed and continuos lines differentiate the results drawn from the SED fitting assuming a type 1 and type 2 AGN, respectively. Bottom panel displays the \texttt{SYNTHESIZER-AGN} best-fit (in black) along with the contributions of the nebular and stellar components (in blue), the QSO template (in magenta) and the dusty torus (in brown).  (*fig:SED_AGNs*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.02997"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

139  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

16  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

7  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
